In [19]:
import keras
import keras.backend as K
import numpy as np

import utils
import loggingreporter 

from collections import namedtuple

cfg = {}
cfg['SGD_BATCHSIZE']    = 128
cfg['SGD_LEARNINGRATE'] = 0.001
cfg['NUM_EPOCHS']       = 20

cfg['ACTIVATION'] = 'relu'
#cfg['ACTIVATION'] = 'tanh'
# How many hidden neurons to put into each of the layers
cfg['LAYER_DIMS'] = [1024, 20, 20, 20]
#cfg['LAYER_DIMS'] = [32, 28, 24, 20, 16, 12, 8, 8]
#cfg['LAYER_DIMS'] = [128, 64, 32, 16, 16] # 0.967 w. 128
#cfg['LAYER_DIMS'] = [20, 20, 20, 20, 20, 20] # 0.967 w. 128
#ARCH_NAME =  '-'.join(map(str,cfg['LAYER_DIMS']))
ARCH_NAME = 'simple_CNN'
# trn, tst = utils.get_mnist()
nb_classes = 10
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train = np.reshape(X_train, [X_train.shape[0], 28, 28, 1]).astype('float32') / 255.
X_test  = np.reshape(X_test , [X_test.shape[0] , 28, 28, 1]).astype('float32') / 255.
Y_train = keras.utils.np_utils.to_categorical(y_train, nb_classes).astype('float32')
Y_test  = keras.utils.np_utils.to_categorical(y_test, nb_classes).astype('float32')

Dataset = namedtuple('Dataset',['X','Y','y','nb_classes'])
trn = Dataset(X_train, Y_train, y_train, nb_classes)
tst = Dataset(X_test , Y_test, y_test, nb_classes)
# Where to save activation and weights data
cfg['SAVE_DIR'] = 'rawdata/' + cfg['ACTIVATION'] + '_' + ARCH_NAME 

In [20]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam

In [21]:
#input_layer  = keras.layers.Input((trn.X.shape[1],))
#clayer = input_layer
#for n in cfg['LAYER_DIMS']:
#    clayer = keras.layers.Dense(n, activation=cfg['ACTIVATION'])(clayer)
#output_layer = keras.layers.Dense(trn.nb_classes, activation='softmax')(clayer)

#model = keras.models.Model(inputs=input_layer, outputs=output_layer)
#optimizer = keras.optimizers.SGD(lr=cfg['SGD_LEARNINGRATE'])

#model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

cnn1 = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=[28, 28, 1]),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])
cnn1.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])

In [67]:
trn.y.shape

(60000,)

In [22]:
num_classes = 10
model3=Sequential()
model3.add(Conv2D(filters=16, kernel_size=(3, 3), padding="same", 
          input_shape=X_train.shape[1:], activation='relu'))
#model3.add(Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation='relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Dropout(0.5))
#model3.add(Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation='relu'))
#model3.add(Conv2D(filters=256, kernel_size=(3, 3), padding="valid", activation='relu'))
#model3.add(MaxPooling2D(pool_size=(3, 3)))
#model3.add(Dropout(0.5))
model3.add(Flatten())
#model3.add(Dense(256))
#model3.add(LeakyReLU())
#model3.add(Dropout(0.5))
model3.add(Dense(32))
#model3.add(LeakyReLU())
#model2.add(Dropout(0.5))
model3.add(Dense(num_classes, activation='softmax'))
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 14, 14, 16)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 32)                100384    
_________________________________________________________________
dense_14 (Dense)             (None, 10)                330       
Total params: 100,874
Trainable params: 100,874
Non-trainable params: 0
_________________________________________________________________


In [23]:
def do_report(epoch):
    # Only log activity for some epochs.  Mainly this is to make things run faster.
    if epoch < 20:       # Log for all first 20 epochs
        return True
    elif epoch < 100:    # Then for every 5th epoch
        return (epoch % 5 == 0)
    elif epoch < 200:    # Then every 10th
        return (epoch % 10 == 0)
    else:                # Then every 100th
        return (epoch % 100 == 0)
    
reporter = loggingreporter.LoggingReporter(cfg=cfg, 
                                          trn=trn, 
                                          tst=tst, 
                                          do_save_func=do_report)
r = model3.fit(x=trn.X, y=trn.Y, 
              verbose    = 1, 
              batch_size = cfg['SGD_BATCHSIZE'],
              epochs     = cfg['NUM_EPOCHS'],
              # validation_data=(tst.X, tst.Y),
              callbacks  = [reporter,])


Epoch 1/20
 1664/60000 [..............................] - ETA: 18s - loss: 1.6801 - acc: 0.4387

/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/keras/callbacks.py:97: UserWarning: Method on_batch_begin() is slow compared to the batch update (0.130404). Check your callbacks.
  % delta_t_median)


60000/60000 [==============================] - 7s 122us/step - loss: 0.5660 - acc: 0.7970
Saving rawdata/relu_simple_CNN/epoch00000000
Epoch 2/20
60000/60000 [==============================] - 7s 115us/step - loss: 0.4117 - acc: 0.8536
Saving rawdata/relu_simple_CNN/epoch00000001
Epoch 3/20
60000/60000 [==============================] - 6s 99us/step - loss: 0.3782 - acc: 0.8668
Saving rawdata/relu_simple_CNN/epoch00000002
Epoch 4/20
60000/60000 [==============================] - 6s 99us/step - loss: 0.3584 - acc: 0.8726
Saving rawdata/relu_simple_CNN/epoch00000003
Epoch 5/20
60000/60000 [==============================] - 6s 102us/step - loss: 0.3451 - acc: 0.8771
Saving rawdata/relu_simple_CNN/epoch00000004
Epoch 6/20
60000/60000 [==============================] - 6s 99us/step - loss: 0.3346 - acc: 0.8810
Saving rawdata/relu_simple_CNN/epoch00000005
Epoch 7/20
60000/60000 [==============================] - 6s 99us/step - loss: 0.3234 - acc: 0.8847
Saving rawdata/relu_simple_CNN/epoch00

In [72]:
cnn1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 5408)              0         
_________________________________________________________________
dense_22 (Dense)             (None, 128)               692352    
_________________________________________________________________
dense_23 (Dense)             (None, 10)                1290      
Total params: 693,962
Trainable params: 693,962
Non-trainable params: 0
_________________________________________________________________


In [24]:
score = model3.evaluate(tst.X, tst.Y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

('Test loss:', 0.2805305122733116)
('Test accuracy:', 0.9022)
